# import new/updated characteristics from google sheet file exported as csv


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

In [5]:
import csv
csv_file_path='./20200608 SEARCH-by strat. representation-Bin. obs_v1 - MiCorr.csv'
with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    print(reader.fieldnames)

def to_uid(name):
    camel_case_name=''.join(name.title().split(' '))

    return camel_case_name[0].lower() + camel_case_name[1:]


['Characteristic', 'Sub-characteristic', 'Observation mode', 'Values', 'Sub-values', 'Image', 'Variables', 'uid', 'Definition from observation under binocular', 'Definition from observation on cross-section', 'Notice for modification of the current MiCorr interface', 'Comments', 'CP Binocular', 'CP CS ', 'CM Binocular', 'CM CS ', 'D Binocular', 'D CS', 'M Binocular', 'M CS', 'NMM Binocular', 'NMM CS', 'POM Binocular', 'POM CS', 'S Binocular', 'S CS', 'SV Binocular', 'SV CS']


In [7]:


with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    current_value=''
    characteristic_order = 1
    family_order = 1

    description_field_in = {'Binocular':'Definition from observation under binocular',
                           'Cross-section': 'Definition from observation on cross-section'}
    for r in reader:
        if 'Microstructure' not in r['Characteristic']:
            continue
        observation = r['Observation observation']
        if r['Values'] != '':
            print('\nNew familly:\n')
            current_value = r['Values']
            current_value_uid = to_uid(current_value)+'Family'
            cql_family_update_row=f"""{{fg:'fgMicrostructure', order:{family_order}, observation:'{observation}',uid:'{current_value_uid}', name:{current_value}, visible: false, variable: false, description:'{r[description_field_in[observation]]}'}},"""
            print(cql_family_update_row)
            family_order+=1
            characteristic_order = 1

        # print(observation, r['Characteristic'], current_value,'None' if r['Sub-values'] == '' else r['Sub-values'])
        current_subvalue = r['Sub-values']
        if observation=='Binocular' and current_value!='' and current_subvalue!='':
            if characteristic_order==1:
                print(f'// {current_value_uid} Sub-Characteristics:')
            current_subvalue_uid = to_uid(current_value + ' `' +current_subvalue)+'Characteristic'
            cql_update_row = f"""{{family_uid:"{current_value_uid}", uid:"{current_subvalue_uid}", name:"{current_subvalue_uid}", order:{characteristic_order},
""" # description:"{r[description_field_in[observation]]}", image_url:""}},
            print(cql_update_row)
            characteristic_order+=1




KeyError: 'Observation observation'

# import image urls from html export


In [4]:
from bs4 import BeautifulSoup
f=open('20200608 SEARCH-by strat. representation-Bin. obs_v1/MiCorr.html')
content=f.read()
soup=BeautifulSoup(content,'html.parser')
imgs=soup.find_all('img')
name_img_url={}
for im in imgs:
    parent_td=im.parent.parent if im.parent.parent.name=='td' else None
    if parent_td:
        name_img_url[parent_td.previous_sibling.previous_sibling.text]=im['src']

In [5]:
print(name_img_url)



{'Dishing': 'https://lh5.googleusercontent.com/OjbxPEa4xZc_YncCe7RM3SzrhZN0k5mjfkRVknk9Vk3UqBtLdU0E4sSzgTyols1Rt_-y7JmnxubX6JshjLA4JQQVKV6RXWWI2vNIhs4hWceDfrC9sjeqlQ0O72R7PqZO3RtZ5jhmTw=w126-h123', 'Flaking': 'https://lh3.googleusercontent.com/vr9yZY_oEuhR5nhmuahX4rWQQtOvVlRg-PAeE2IaCbYgqXvDnm1x1EEvuFRrhmLxNcTPRSR__g1SlUalVG-bWsQs9TUSTuH1AL5NObldMoxAdt0T6JQXZ051cMybyLy9hGnNCR2HjA=w126-h112', 'Wedges': 'https://lh3.googleusercontent.com/jsgPu7PBzPb1le0GSLN-B47anat1-zwLQbUcnO7Nxc1a8Ynfw0Yb3ibM0vxKd6Jf8UePE_H4JXbASZ_cn4m-UhT71lbcbJlWOE4RQSccpXiOBBp--Y2LC1pxvWbmG1DZC12yO5Io-A=w126-h137', 'Surface crazing': 'https://lh5.googleusercontent.com/s0Itc5SPXPvI5RgR39MOZi0I6PUNeNI7BDUWV6GSEnnBmbNXtRxAXwLzm578PhGUM9iNX07JdztbLfm43kM29DKRb9SpiCdFbGGAzm4ynOK7RP6pIFfYmXtOS1uG8wJptef3FV0Fvg=w126-h128', 'Star-shaped cracking': 'https://lh4.googleusercontent.com/0y0Sww61R3c5CkqHoT1JADz3VYUEw2EWGTndn3WfKb--XNC4LZhLHXeneeyf3-e0h7IdAhK7M-b_hIZ7cPp5Y3aJpA9o8pfXB-hy_bTYSJwUZNQnKyllGXyVmWKpqQL7EUcEh8Ok3w=w126-h

In [ ]:
## Microstructure Binocular special case
# we treat all values and sub-values as characteristics (with optgroup + property for Cristalline microstructure)
# and optgroup - (closure of previous group) for isolated aggregate

def new_characteristic(family_uid, uid, name, order, description=None, image_url=None, optgroup=None):
    cql_update_row = f"""{{family_uid:"{family_uid}", uid:"{uid}", name:"{name}", order:{order}"""
    arg_list=[cql_update_row]

    if image_url is None:
        # search characteristic name in name to image_url map
        image_url = name_img_url.get(name)
    if description:
        arg_list.append(f'description="{description}"')
    if image_url:
        arg_list.append(f'image_url="{image_url}"')
    if optgroup:
        arg_list.append(f'optgroup="{optgroup}"')

    return ', '.join(arg_list) + '},'

def new_family(fg_uid, uid, name, order, observation, description='',visible=False, variable=False):
    return f"""{{fg:'{fg_uid}', order:{order}, observation:{observation}, uid:'{uid}', name:{name}, visible: {visible}, variable: {variable}, description:'{description}}},"""


with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    current_value=''
    characteristic_order = 1
    family_order = 1
    sub_value_order = 1
    optgroup_level = 0
    description_field_in = {'Binocular':'Definition from observation under binocular',
                           'Cross-section': 'Definition from observation on cross-section'}

    for r in reader:
        if 'Microstructure' not in r['Characteristic']:
            continue
        observation = r['Observation mode']
        if observation != 'Binocular':
            continue

        if r['Values'] == '' and r['Sub-values'] == '':
            family_uid='microstructureFamily'
            cql_family_update_row=new_family(fg_uid='fgMicrostructure', uid=family_uid, name='Microstructure',
                                     order=family_order,observation='binocular', description=r[description_field_in[observation]])
            print(cql_family_update_row)
            print(f'\n// {family_uid} characteristics:')
            family_order+=1

        if r['Values'] != '':
            # print('\nNew characteristic:\n')
            current_value = r['Values']
            current_value_uid = to_uid(r['Characteristic'] + ' ' +current_value)+'Characteristic'
            # trick to detect end of optgroup in our particular case (1 optgroup only yet)
            if optgroup_level:
                optgroup=None
                optgroup_level-=1

            print(new_characteristic(family_uid=family_uid, uid=current_value_uid,
                                     name=current_value,order=characteristic_order,
                                     description=r[description_field_in[observation]],optgroup=optgroup))
            # family_order+=1
            # characteristic_order = 1
            characteristic_order += 1

        # print(observation, r['Characteristic'], current_value,'None' if r['Sub-values'] == '' else r['Sub-values'])
        current_subvalue = r['Sub-values']
        if observation=='Binocular' and current_value!='' and current_subvalue!='':
            if sub_value_order==1:
                optgroup_level+=1
                optgroup=current_value
                print(f'// {current_value_uid} optgroup options:')
            current_subvalue_uid = to_uid(current_value + ' ' +current_subvalue)+'Characteristic'
            print(new_characteristic(family_uid=family_uid, uid=current_subvalue_uid,name=current_subvalue,
                                     order=characteristic_order))
            sub_value_order+=1
            characteristic_order+=1

In [9]:
## Microstructure Binocular special case
# we treat all values and sub-values as characteristics (with optgroup + property for Cristalline microstructure)
# and optgroup - (closure of previous group) for isolated aggregate

def new_characteristic(family_uid, uid, name, order, description=None, image_url=None, optgroup=None):
    cql_update_row = f"""{{family_uid:"{family_uid}", uid:"{uid}", name:"{name}", order:{order}"""
    arg_list=[cql_update_row]

    if image_url is None:
        # search characteristic name in name to image_url map
        image_url = name_img_url.get(name)
    if description:
        arg_list.append(f'description:"{description}"')
    if image_url:
        arg_list.append(f'image_url:"{image_url}"')
    if optgroup:
        arg_list.append(f'optgroup:"{optgroup}"')

    return ', '.join(arg_list) + '},'

def new_family(fg_uid, uid, name, order, observation, description='',visible=False, variable=False):
    return f"""{{fg:'{fg_uid}', order:{order}, observation:{observation}, uid:'{uid}', name:'{name}', visible: {visible}, variable: {variable}, description:'{description}}},"""


with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    current_value=''
    characteristic_order = 1
    family_order = 1
    sub_value_order = 1
    optgroup = None
    optgroup_level = 0
    description_field_in = {'Binocular':'Definition from observation under binocular',
                           'Cross-section': 'Definition from observation on cross-section'}

    for r in reader:
        if 'Microstructure' not in r['Characteristic']:
            continue
        observation = r['Observation mode']
        if observation != 'Binocular':
            continue

        if r['Values'] == '' and r['Sub-values'] == '':
            family_uid='microstructureFamily'
            cql_family_update_row=new_family(fg_uid='fgMicrostructure', uid=family_uid, name='Microstructure',
                                     order=family_order,observation='binocular', description=r[description_field_in[observation]])
            print(cql_family_update_row)
            print(f'\n// {family_uid} characteristics:')
            family_order+=1

        if r['Values'] != '':
            # print('\nNew characteristic:\n')
            current_value = r['Values']
            current_value_uid = to_uid(r['Characteristic'] + ' ' +current_value)+'Characteristic'
            # trick to detect end of optgroup in our particular case (1 optgroup only yet)

            if optgroup_level:
                optgroup= None
                optgroup_level-=1

            print(new_characteristic(family_uid=family_uid, uid=current_value_uid,
                                     name=current_value,order=characteristic_order,
                                     description=r[description_field_in[observation]],optgroup=optgroup))
            # family_order+=1
            # characteristic_order = 1
            characteristic_order += 1

        # print(observation, r['Characteristic'], current_value,'None' if r['Sub-values'] == '' else r['Sub-values'])
        current_subvalue = r['Sub-values']
        if observation=='Binocular' and current_value!='' and current_subvalue!='':
            if sub_value_order==1:
                optgroup = current_value
                optgroup_level+=1
                print(f'// {current_value_uid} optgroup options:')
            current_subvalue_uid = to_uid(current_value + ' ' +current_subvalue)+'Characteristic'
            print(new_characteristic(family_uid=family_uid, uid=current_subvalue_uid,
                                     name=current_subvalue, order=characteristic_order,
                                     description=r[description_field_in[observation]], optgroup=optgroup))
            sub_value_order+=1
            characteristic_order+=1

{fg:'fgMicrostructure', order:1, observation:binocular, uid:'microstructureFamily', name:'Microstructure', visible: False, variable: False, description:'Assessed on the observed area. The blank option corresponds to a stratum which microstructure is not known.},

// microstructureFamily characteristics:
{family_uid:"microstructureFamily", uid:"microstructureBlankCharacteristic", name:"Blank", order:1},
{family_uid:"microstructureFamily", uid:"microstructureAlternativeBandsCharacteristic", name:"Alternative bands", order:2, description:"Occurrence of several alternately repeating layers of corrosion products of different composition."},
{family_uid:"microstructureFamily", uid:"microstructureCristallineMicrostructureCharacteristic", name:"Cristalline microstructure", order:3, description:"Presence of a crystalline aggregates. The blank option corresponds to a stratum which cristalline microstructure is not known."},
// microstructureCristallineMicrostructureCharacteristic optgroup option